# Load Tensors

gradient clipping, learning rate scheduling, advanced regularization techniques,
 Bidirectional RNNs, attention mechanisms, transformer architectures, oscillators

In [6]:
import torch

# List of tensor paths
tensor_paths = [
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/EEG_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/arnold_tongues_rotation_numbers_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/bandpowers_embedding_net.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/band_power_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/cnn_dspm_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/fast_fourier_transform_psd_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/higuchi_fractal_dimensions_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/Hurst_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_concatd_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/mfdfa_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/short_time_fourier_transform_tensor.pth",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_granular_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_hemispheric_avg_input_tensor.pt",
    "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/CNN/transfer_entropy_regional_tensor.pt"
]

# Load the tensors into a dictionary
tensors = {path.split('/')[-1].replace('.pt', ''): torch.load(path) for path in tensor_paths}

In [7]:
# Load each tensor and print its shape
tensor_shapes = {}
for path in tensor_paths:
    tensor_name = path.split("/")[-1].replace(".pt", "").replace(".pth", "")
    data = torch.load(path)
    
    # Check if the loaded data is a state dictionary or a tensor
    if isinstance(data, torch.Tensor):
        tensor_shapes[tensor_name] = data.shape
    elif isinstance(data, dict):  # Likely a state_dict
        tensor_shapes[tensor_name] = "state_dict (model parameters)"
    else:
        tensor_shapes[tensor_name] = "unknown type"

# Print the tensor shapes
for name, shape in tensor_shapes.items():
    print(f"{name}: {shape}")

EEG_tensorh: torch.Size([1, 4227788, 1, 32])
arnold_tongues_rotation_numbers_tensor: torch.Size([32, 300, 300])
bandpowers_embedding_neth: state_dict (model parameters)
band_power_tensorh: torch.Size([4227788, 32, 5])
cnn_dspm_tensor: torch.Size([19, 18840, 10])
fast_fourier_transform_psd_tensorh: torch.Size([32, 4227788])
higuchi_fractal_dimensions_tensor: torch.Size([1, 1, 4, 8])
Hurst_tensorh: torch.Size([1, 1, 32, 1])
mfdfa_concatd_tensorh: torch.Size([32, 1, 30, 2])
mfdfa_tensorh: torch.Size([9, 32, 10, 1])
short_time_fourier_transform_tensorh: torch.Size([32, 1001, 4229])
transfer_entropy_granular_tensor: torch.Size([4, 4])
transfer_entropy_hemispheric_avg_input_tensor: torch.Size([92, 92])
transfer_entropy_regional_tensor: torch.Size([4, 4])


# CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BaseEmbeddingNet(nn.Module):
    def __init__(self, input_channels, conv_output_channels, height, width):
        super(BaseEmbeddingNet, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, conv_output_channels, 3)
        self.batch_norm = nn.BatchNorm2d(conv_output_channels)
        self.fc1 = nn.Linear(conv_output_channels * (height - 2) * (width - 2), 512)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.batch_norm(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(self.dropout(x)))
        return x

class EEGEmbeddingNet(BaseEmbeddingNet):
    def __init__(self, num_timepoints):
        super().__init__(32, 64, num_timepoints, 1)

class RotationEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 298, 298)

class BandPowerEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 5, 5)

class DSPMEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(19, 64, 18840, 10)

class FastFourierEmbeddingNet(BaseEmbeddingNet):
    def __init__(self, new_dim_height, new_dim_width):
        super().__init__(32, 64, new_dim_height, new_dim_width)

class HiguchiFractalEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 4, 8)

class HurstEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 32, 1)

class MFDFAConcatdEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1, 64, 30, 2)

class MFDFAEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(32, 64, 10, 1)

class ShortTimeFourierEmbeddingNet(BaseEmbeddingNet):
    def __init__(self):
        super().__init__(1001, 64, 4229, 1)

class PairwiseMeasureNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, 1)
    
    def forward(self, x):
        return self.conv(x)


In [ ]:

# Main Network
class MainNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Assuming all embedding networks produce a 512 sized vector
        # If we have 10 tensors, then 10*512 = 5120
        self.fc1 = nn.Linear(5120, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 1) # Assuming binary classification

    def forward(self, *args):
        # Assuming each tensor is processed through its embedding network
        # and then passed in order as arguments to this forward method
        concatenated = torch.cat(args, dim=1)
        x = self.fc1(concatenated)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


# Spectral Centroid Branch

In [ ]:
import numpy as np
import torch

# Convert dictionary values to numpy array
spectral_centroids_array = np.array(list(spectral_centroids_data.values()))

# Convert numpy array to PyTorch tensor
spectral_centroids_tensor = torch.tensor(spectral_centroids_array, dtype=torch.float32).unsqueeze(0)

# Print out the shape to confirm
print(spectral_centroids_tensor.shape)  # This should print torch.Size([1, 32])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNEEGPredictor(nn.Module):
    def __init__(self, num_channels, num_time_points, num_centroids):
        super(CNNEEGPredictor, self).__init__()
        # Assuming a 1D CNN here for simplicity
        self.conv1 = nn.Conv1d(num_channels, 64, kernel_size=5)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5)
        self.fc1 = nn.Linear(128 * num_time_points + num_centroids, 256)
        self.fc2 = nn.Linear(256, num_time_points)  # Adjust the output size as needed

    def forward(self, x, centroids):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the data
        x = torch.cat([x, centroids], dim=1)  # Concatenate the flattened data with centroids
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
